In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from pandas import read_csv

import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")
def conlog(row):
    
    return np.log1p(row)

def invlog(row):
    return np.expm1(row)


In [ ]:
trainfile="../input/train.tsv"
testfile="../input/test.tsv"
output=""

#trainfile="C:/Users/nsrikaku/Desktop/Kaggle/mer/train.tsv"
#testfile="C:/Users/nsrikaku/Desktop/Kaggle/mer/test.tsv"
#output="C:/Users/nsrikaku/Desktop/Kaggle/mer/submissions/"

In [ ]:
dataset=pd.read_csv(trainfile,delimiter="\t")
dataset.set_index('train_id',inplace=True)


In [ ]:
X=dataset[['item_condition_id','shipping']]
Y=conlog(dataset["price"])


In [ ]:
X["brand_null"]=dataset['brand_name'].isnull()
X["des_null"]=dataset['item_description']=='No description yet'
del dataset

In [ ]:
model = XGBRegressor()


#learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
max_depth = [2,5,10]
#[1,3,5,7,9]
learning_rate = [0.1,0.2]
n_estimators = range(50, 100, 50)


#Tuning Row Subsampling without replacement
subsample = [0.6,1.0]

#Tuning Column Subsampling By Tree
colsample_bytree = [0.8,1.0]

#Tuning Column Subsampling By Split
colsample_bylevel = [0.8, 1.0]

#minsamplessplit = 2
param_grid = dict(n_estimators=n_estimators,learning_rate=learning_rate,max_depth=max_depth,subsample=subsample,colsample_bytree=colsample_bytree,colsample_bylevel=colsample_bylevel)
kfold = KFold(n_splits=2, shuffle=True, random_state=7)

In [ ]:

#scoring="accuracy"
grid_search = GridSearchCV(model, param_grid,n_jobs=5, cv=kfold,scoring='neg_mean_squared_error')
grid_result = grid_search.fit(X, Y)

In [ ]:
print(grid_result.best_score_)
model=grid_result.best_estimator_

In [ ]:
test_pre=pd.read_csv(testfile,delimiter="\t")
test_pre.set_index('test_id',inplace=True)


In [ ]:

test_pre=pd.read_csv(testfile,delimiter="\t")
test_pre.set_index('test_id',inplace=True)
test_feat=test_pre[['item_condition_id','shipping']]
test_feat["brand_null"]=test_pre['brand_name'].isnull()
test_feat["des_null"]=test_pre['item_description']=='No description yet'
test_feat["price"]=invlog(model.predict(test_feat))
test_feat[["price"]].to_csv(output+"sub_mer_xgboost_log.csv")